In [1]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import os

# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, label_binarize
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, balanced_accuracy_score, roc_auc_score

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [2]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [3]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),유지방율,유단백,무지고형분,체세포수,MUN,305일유량,...,공태일수,유량,질병군,우유등급,비유초기평균MUN.1,비유최고도달일수_log,건유전마지막유량_log,공태일수_log,출생년도,PF비
0,21133,2.007121e+13,2021-03-18,445,6.60,3.28,8.87,274,11.6,15158.0,...,619,76169939,0,2등급이하,NaN,4.143135,4.330733,6.429719,2007,0.496970
1,21133,2.007121e+13,2021-04-28,486,3.64,3.33,8.91,166,11.0,15158.0,...,619,76170244,0,1등급,NaN,4.143135,4.330733,6.429719,2007,0.914835
2,21133,2.007121e+13,2021-05-20,508,2.61,3.35,9.14,156,10.8,15158.0,...,619,76170487,0,1등급,NaN,4.143135,4.330733,6.429719,2007,1.283525
3,21133,2.007121e+13,2021-06-25,544,3.75,3.49,8.99,307,14.1,15158.0,...,619,76170734,0,2등급이하,NaN,4.143135,4.330733,6.429719,2007,0.930667
4,21133,2.007121e+13,2021-07-07,556,3.68,3.74,9.31,34,16.2,15158.0,...,619,76170903,0,1등급,NaN,4.143135,4.330733,6.429719,2007,1.016304


In [4]:
df = milk

In [5]:
df['1등급'] = df['우유등급'].map({
    '1등급': 1,
    '2등급이하': 0
})

In [6]:
target='1등급'
features = [
    '누적착유일(연계)',
    '전산차비유지속성',
    '현재산차비유지속성',
    '비유최고도달일수_log',
    '건유전마지막유량_log',
    '전산차건유전유량',
    '산차',
    '농후사료급여량(연계)',
    '공태일수_log'
]

In [7]:
train = df[df['검정일자'].dt.year == 2020]
test = df[df['검정일자'].dt.year == 2021]

X_train, X_test = train[features], test[features]
y_train, y_test = train['1등급'], test['1등급']

In [8]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import time
import os
import json
import warnings
warnings.filterwarnings("ignore")

# ============================================
# 1. 클래스 비율 확인
# ============================================
print("="*60)
print("클래스 분포 분석")
print("="*60)
print(y_train.value_counts().sort_index())
print(f"\n비율:")
print(y_train.value_counts(normalize=True).sort_index())

# 🔥 수정: 0이 소수, 1이 다수
minority_count = (y_train == 0).sum()  # 그외 (소수)
majority_count = (y_train == 1).sum()  # 1등급 (다수)

print(f"\n클래스 구성:")
print(f"   0 (그외, 소수): {minority_count}개 ({minority_count/len(y_train)*100:.1f}%)")
print(f"   1 (1등급, 다수): {majority_count}개 ({majority_count/len(y_train)*100:.1f}%)")

scale_pos_weight = minority_count / majority_count
print(f"\n계산된 비율 (소수/다수): {scale_pos_weight:.2f}")

# 결측치 확인
print("\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train.columns,
    '결측_개수': X_train.isnull().sum().values,
    '결측_비율(%)': (X_train.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info)
print(f"\n전체 결측 비율: {X_train.isnull().mean().mean()*100:.1f}%")
print(f"Feature 개수: {X_train.shape[1]}개")

# ============================================
# 2. Pipeline 구성
# ============================================
pipeline = ImbPipeline([
    ('imputer', SimpleImputer(
        strategy='median',
        add_indicator=True
    )),
    
    ('smote', SMOTE(
        random_state=42
    )),
    
    ('model', LGBMClassifier(
        objective='binary',
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=1
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. SimpleImputer (median + missing indicator)")
print("   2. SMOTE (소수 클래스 0 증강)")
print("   3. LGBMClassifier (GPU)")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(0) 샘플 수: {minority_count}개")

max_k = min(7, minority_count - 1)
print(f"SMOTE k_neighbors 최대값: {max_k}")

from scipy.stats import uniform, randint

param_distributions = {
    # 🔥 SMOTE 파라미터 - 더 공격적으로
    'smote__sampling_strategy': [0.7, 0.9, 1.0],  # 높은 비율로 증강
    'smote__k_neighbors': randint(3, max_k + 1),
    
    # 🔥 LightGBM 파라미터 - 소수 클래스 민감도 향상
    'model__n_estimators': randint(300, 1000),     # 더 많은 트리
    'model__learning_rate': uniform(0.005, 0.045), # 더 작은 학습률 (0.005~0.05)
    'model__max_depth': randint(3, 8),
    'model__num_leaves': randint(15, 80),
    'model__min_child_samples': randint(5, 50),    # 🔥 더 작게 (소수 클래스 민감)
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__min_child_weight': [1e-5, 1e-4, 1e-3, 1e-2],  # 🔥 더 작게
    'model__reg_alpha': uniform(0, 0.5),           # L1 정규화 줄임
    'model__reg_lambda': uniform(0, 0.5),          # L2 정규화 줄임
    'model__scale_pos_weight': uniform(0.5, 2.5)   # 🔥 추가: 0.5~3.0
}

# ============================================
# 4. Scoring 설정 (🔥 f1_macro 메인)
# ============================================
from sklearn.metrics import make_scorer, recall_score, precision_score, f1_score, fbeta_score

# 🔥 F2 score: Recall을 Precision보다 2배 중요하게
f2_scorer = make_scorer(fbeta_score, beta=2, pos_label=0, zero_division=0)

scoring = {
    'recall': make_scorer(recall_score, pos_label=0),  # 🔥 메인 목표
    'f2': f2_scorer,                                    # Recall 가중
    'precision': make_scorer(precision_score, pos_label=0, zero_division=0),
    'f1': make_scorer(f1_score, pos_label=0, zero_division=0),
    'pr_auc': make_scorer(
        lambda y_true, y_pred: __import__('sklearn.metrics').metrics.average_precision_score(
            y_true, y_pred, pos_label=0
        ),
        needs_proba=True
    ),
    'roc_auc': 'roc_auc'
}


# ============================================
# 5. Stratified K-Fold
# ============================================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_folds = 5
n_iter = 150

print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. 예상 시간 계산
# ============================================
print("\n" + "="*60)
print("⏱️  예상 시간 계산 중...")
print("="*60)

sample_pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='median', add_indicator=True)),
    ('smote', SMOTE(random_state=42, sampling_strategy=0.5, k_neighbors=3)),
    ('model', LGBMClassifier(
        objective='binary',
        n_estimators=300,
        learning_rate=0.05,
        max_depth=5,
        num_leaves=31,
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=1
    ))
])

start_time = time.time()
_ = cross_val_score(
    sample_pipeline, X_train, y_train,
    cv=3,
    scoring='f1_macro',
    n_jobs=1
)
sample_time = time.time() - start_time

n_cores = os.cpu_count() or 4
estimated_per_combination = sample_time / 3 * n_folds
estimated_total_sequential = estimated_per_combination * n_iter
parallel_factor = min(2.5, n_cores / 2)
estimated_parallel = estimated_total_sequential / parallel_factor

print(f"샘플 Pipeline 소요: {sample_time:.1f}초 (3-fold)")
print(f"조합당 예상 시간: {estimated_per_combination:.1f}초 (5-fold)")
print(f"예상 총 시간 (순차): {estimated_total_sequential/60:.1f}분")
print(f"예상 총 시간 (병렬, 효율 {parallel_factor:.1f}x): {estimated_parallel/60:.1f}분")

estimated_finish = time.time() + estimated_parallel
print(f"\n💡 예상 완료 시각: {time.strftime('%H:%M:%S', time.localtime(estimated_finish))}")

if estimated_parallel < 1800:
    print(f"\n✅ 예상 시간이 적절합니다 (~{estimated_parallel/60:.0f}분)")
elif estimated_parallel < 3600:
    print(f"\n⚠️  예상 시간이 다소 깁니다 (~{estimated_parallel/60:.0f}분)")
    print("   커피 한 잔 하고 오시면 됩니다 ☕")
else:
    print(f"\n⚠️  예상 시간이 깁니다 (~{estimated_parallel/60:.0f}분)")
    print(f"   n_iter를 줄이는 것을 고려하세요")

# ============================================
# 7. RandomizedSearchCV 실행
# ============================================
print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit='recall',  # 🔥 F1-macro 기준으로 최적 모델 선택
    verbose=2,
    random_state=42,
    n_jobs=-1,
    # return_train_score=False
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 8. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  실제 소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")
print(f"   예상 대비: {actual_time/estimated_parallel*100:.0f}%")

if actual_time < estimated_parallel * 0.8:
    print("   → 예상보다 빠름! 병렬 처리 효율 좋음 ✨")
elif actual_time > estimated_parallel * 1.2:
    print("   → 예상보다 느림. 시스템 부하 확인 필요")
else:
    print("   → 예상 시간과 유사함")

print(f"\n🏆 최고 Recall (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")

# ============================================
# 9. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

comparison_cols = [
    'rank_test_recall',
    'mean_test_recall',
    'mean_test_f2',
    'mean_test_precision',
    'mean_test_f1',
    'mean_test_pr_auc',
    'mean_test_roc_auc',
    'params'
]

print("\n" + "="*60)
print("📊 상위 10개 파라미터 조합 (Recall 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values('rank_test_recall').head(10)

for idx, row in top_10.iterrows():
    print(f"\n#{int(row['rank_test_recall'])} - Recall: {row['mean_test_recall']:.4f}")
    print(f"   F2: {row['mean_test_f2']:.3f} | Precision: {row['mean_test_precision']:.3f} | F1: {row['mean_test_f1']:.3f}")
    
    params = row['params']
    print(f"   SMOTE: strategy={params.get('smote__sampling_strategy', 'N/A')}, k={params.get('smote__k_neighbors', 'N/A')}")
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, depth={params.get('model__max_depth', 0)}, leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 10. SMOTE 효과 분석
# ============================================
print("\n" + "="*60)
print("🔬 SMOTE 효과 분석")
print("="*60)

best_sampling = lgbm_random.best_params_.get('smote__sampling_strategy', 1.0)
best_k = lgbm_random.best_params_.get('smote__k_neighbors', 5)

minority_after_smote = int(majority_count * best_sampling)

print(f"원본 데이터:")
print(f"   1 (1등급, 다수): {majority_count}개")
print(f"   0 (그외, 소수): {minority_count}개")
print(f"   비율: {majority_count/minority_count:.1f}:1")

print(f"\nSMOTE 적용 후 (최적 파라미터):")
print(f"   sampling_strategy: {best_sampling}")
print(f"   k_neighbors: {best_k}")
print(f"   1 (1등급): {majority_count}개 (유지)")
print(f"   0 (그외): ~{minority_after_smote}개 (증강)")
print(f"   비율: ~{1/best_sampling:.1f}:1")
print(f"   증강된 샘플: {minority_after_smote - minority_count}개")

# ============================================
# 11. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

y_pred_proba_minority = y_pred_proba[:, 0]

print(classification_report(y_test, y_pred, target_names=['0 (그외, 소수)', '1 (1등급, 다수)']))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (그외)', '실제_1 (1등급)'],
                     columns=['예측_0 (그외)', '예측_1 (1등급)'])
print(cm_df)

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=0=그외 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (1→0): {fp}개 → 1등급을 그외로 오판 (재검사 필요)")
print(f"   🚨 미탐 (0→1): {fn}개 → 그외를 1등급으로 오판 (치명적!)")

if tn + fn > 0:
    recall_pct = tn/(tn+fn)*100
    print(f"   📊 소수 클래스(0) Recall: {recall_pct:.1f}% (그외 {tn+fn}개 중 {tn}개 검출)")
    print(f"      → 이전 14% 대비 개선: {recall_pct - 14:.1f}%p ✨")
else:
    print(f"   📊 소수 클래스(0) Recall: N/A")

if tn + fp > 0:
    precision_pct = tn/(tn+fp)*100
    print(f"   📊 소수 클래스(0) Precision: {precision_pct:.1f}% (그외 판정 {tn+fp}개 중 {tn}개 정답)")
else:
    print(f"   📊 소수 클래스(0) Precision: N/A")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 12. 🔥 임계값 조정으로 추가 개선
# ============================================
from sklearn.metrics import precision_recall_curve, average_precision_score

precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba_minority, pos_label=0)

# Recall 80% 보장
target_recall = 0.80
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")
    
    y_pred_adjusted = np.where(y_pred_proba_minority >= optimal_threshold, 0, 1)
    
    cm_adj = confusion_matrix(y_test, y_pred_adjusted)
    tn2, fp2, fn2, tp2 = cm_adj.ravel()
    
    print(f"\n조정 효과:")
    print(f"   미탐(FN) 감소: {fn} → {fn2} (개선: {fn-fn2}개)")
    print(f"   오탐(FP) 증가: {fp} → {fp2} (증가: {fp2-fp}개)")
    print(f"   Recall 개선: {tn/(tn+fn)*100:.1f}% → {tn2/(tn2+fn2)*100:.1f}%")
    
    if fn > fn2:
        print(f"   💡 그외를 1등급으로 오판하는 치명적 오류 {fn-fn2}개 감소!")
else:
    print(f"\n⚠️  Recall {target_recall*100}% 달성 불가 (최대 Recall: {recalls.max():.2%})")

# ============================================
# 13. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

imputer = best_model.named_steps['imputer']
if imputer.add_indicator:
    missing_features = X_train.columns[X_train.isnull().any()].tolist()
    for col in missing_features:
        feature_names.append(f"{col}_결측지표")

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

missing_indicators = importance_df[importance_df['feature'].str.contains('결측지표', na=False)]
if len(missing_indicators) > 0:
    print("\n💡 결측 지표 변수 중요도:")
    print(missing_indicators.to_string(index=False))

# ============================================
# 14. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

test_metrics = {
    'Recall': recall_score(y_test, y_pred, pos_label=0),
    'F2': fbeta_score(y_test, y_pred, beta=2, pos_label=0),
    'Precision': precision_score(y_test, y_pred, pos_label=0, zero_division=0),
    'F1': f1_score(y_test, y_pred, pos_label=0, zero_division=0),
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority, pos_label=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['Recall', 'F2', 'Precision', 'F1', 'PR-AUC', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df['rank_test_recall'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 15. 최종 요약
# ============================================
print("\n" + "="*60)
print("📝 최종 요약 (Recall 개선)")
print("="*60)
print(f"✅ 최적 모델 Recall (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}%")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}%")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f}")
print(f"✅ 치명적 오류(미탐): {fn}개 / {tn+fn}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n🎯 개선 효과:")
print(f"   - Recall: 14.0% → {test_metrics['Recall']*100:.1f}% (↑{test_metrics['Recall']*100-14:.1f}%p)")
print(f"   - 미탐 감소: {int(4698*0.86)} → {fn}개 (↓{int(4698*0.86)-fn}개)")

print("\n💡 핵심 포인트:")
print(f"   - 그외(0) {tn+fn}개 중 {tn}개 검출 ({tn/(tn+fn)*100:.1f}%)")
print(f"   - 놓친 그외: {fn}개 (이전 {int(4698*0.86)}개 대비 대폭 감소)")
print(f"   - 오탐 증가: {fp}개 (재검사 필요하지만 감수 가능)")

# ============================================
# 16. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model = input("모델을 저장하시겠습니까? (y/n): ")
if save_model.lower() == 'y':
    import joblib
    from datetime import datetime
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_recall_optimized_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_recall_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_recall_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_recall_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        'best_recall_score': float(lgbm_random.best_score_),
        'optimization_target': 'recall',
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'smote_sampling_strategy': float(best_sampling),
        'smote_k_neighbors': int(best_k)
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        },
        'business_interpretation': {
            'total_minority': int(tn + fn),
            'detected_minority': int(tn),
            'missed_minority': int(fn),
            'recall_percentage': float(tn/(tn+fn)*100),
            'false_alarms': int(fp)
        },
        'improvement': {
            'baseline_recall': 14.0,
            'new_recall': float(test_metrics['Recall']*100),
            'improvement_pp': float(test_metrics['Recall']*100 - 14.0)
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)

클래스 분포 분석
1등급
0     3682
1    26393
Name: count, dtype: int64

비율:
1등급
0    0.122427
1    0.877573
Name: proportion, dtype: float64

클래스 구성:
   0 (그외, 소수): 3682개 (12.2%)
   1 (1등급, 다수): 26393개 (87.8%)

계산된 비율 (소수/다수): 0.14

결측치 현황:
        Feature  결측_개수   결측_비율(%)
4  건유전마지막유량_log  13887  46.174564
5      전산차건유전유량  13887  46.174564
6            산차   2822   9.383209
0     누적착유일(연계)      0   0.000000
1      전산차비유지속성      0   0.000000
3  비유최고도달일수_log      0   0.000000
2     현재산차비유지속성      0   0.000000
7   농후사료급여량(연계)      0   0.000000
8      공태일수_log      0   0.000000

전체 결측 비율: 11.3%
Feature 개수: 9개

✅ Pipeline 구성:
   1. SimpleImputer (median + missing indicator)
   2. SMOTE (소수 클래스 0 증강)
   3. LGBMClassifier (GPU)

소수 클래스(0) 샘플 수: 3682개
SMOTE k_neighbors 최대값: 7

교차검증 Folds: 5
랜덤 샘플링 조합 수: 150개
총 Fits: 150 × 5 = 750

⏱️  예상 시간 계산 중...
샘플 Pipeline 소요: 2.1초 (3-fold)
조합당 예상 시간: 3.4초 (5-fold)
예상 총 시간 (순차): 8.6분
예상 총 시간 (병렬, 효율 2.5x): 3.4분

💡 예상 완료 시각: 16:10:52

✅ 예상 시간이 적절합니다 (~3분)

🚀 Randomi

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=320, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__scale_pos_weight=2.3049969306670617, model__subsample=0.9754210836063001, smote__k_neighbors=4, smote__sampling_strategy=0.7; total time=   6.4s
[CV] END model__colsample_bytree=0.7085396127095583, model__learning_rate=0.04229318791183682, model__max_depth=4, model__min_child_samples=29, model__min_child_weight=0.01, model__n_estimators=487, model__num_leaves=55, model__reg_alpha=0.1481367528520412, model__reg_lambda=0.08263346953150125, model__scale_pos_weight=0.5390910168529848, model__subsample=0.7693605922825478, smote__k_neighbors=3, smote__sampling_strategy=1.0; total time=  54.2s
[CV] END model__colsample_bytree=0.7085396127095583, model__learning_rate=0.04229318791183682, mode

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.7085396127095583, model__learning_rate=0.04229318791183682, model__max_depth=4, model__min_child_samples=29, model__min_child_weight=0.01, model__n_estimators=487, model__num_leaves=55, model__reg_alpha=0.1481367528520412, model__reg_lambda=0.08263346953150125, model__scale_pos_weight=0.5390910168529848, model__subsample=0.7693605922825478, smote__k_neighbors=3, smote__sampling_strategy=1.0; total time=  56.2s
[CV] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=320, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__scale_pos_weight=2.3049969306670617, model__subsample=0.9754210836063001, smote__k_neighbors=4, smote__sampling_strategy=0.7; total time=  55.4s
[CV] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, mode

...

[CV] END model__colsample_bytree=0.9468125984489795, model__learning_rate=0.031040393385757703, model__max_depth=6, model__min_child_samples=38, model__min_child_weight=0.01, model__n_estimators=877, model__num_leaves=48, model__reg_alpha=0.3964780473028158, model__reg_lambda=0.3275046694160543, model__scale_pos_weight=2.980333290851681, model__subsample=0.713541811443458, smote__k_neighbors=7, smote__sampling_strategy=1.0; total time= 3.5min
[CV] END model__colsample_bytree=0.7222097242632094, model__learning_rate=0.022864168268342755, model__max_depth=4, model__min_child_samples=8, model__min_child_weight=0.0001, model__n_estimators=610, model__num_leaves=20, model__reg_alpha=0.4596959866072087, model__reg_lambda=0.24848174145291513, model__scale_pos_weight=2.9803950370582815, model__subsample=0.9405699830980383, smote__k_neighbors=4, smote__sampling_strategy=1.0; total time=   2.9s
[CV] END model__colsample_bytree=0.7222097242632094, model__learning_rate=0.022864168268342755, model_

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.7222097242632094, model__learning_rate=0.022864168268342755, model__max_depth=4, model__min_child_samples=8, model__min_child_weight=0.0001, model__n_estimators=610, model__num_leaves=20, model__reg_alpha=0.4596959866072087, model__reg_lambda=0.24848174145291513, model__scale_pos_weight=2.9803950370582815, model__subsample=0.9405699830980383, smote__k_neighbors=4, smote__sampling_strategy=1.0; total time=   2.9s


[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.741998522332775, model__learning_rate=0.008124072923574201, model__max_depth=3, model__min_child_samples=6, model__min_child_weight=0.01, model__n_estimators=816, model__num_leaves=58, model__reg_alpha=0.11685604095521651, model__reg_lambda=0.2700059573022553, model__scale_pos_weight=2.7001977189036457, model__subsample=0.8603509476188776, smote__k_neighbors=6, smote__sampling_strategy=0.7; total time=  43.5s
[CV] END model__colsample_bytree=0.7022579991237089, model__learning_rate=0.018592722582565776, model__max_depth=5, model__min_child_samples=40, model__min_child_weight=0.01, model__n_estimators=466, model__num_leaves=32, model__reg_alpha=0.383328067600203, model__reg_lambda=0.07485198397847886, model__scale_pos_weight=1.559862032283255, model__subsample=0.6611935203314251, smote__k_neighbors=3, smote__sampling_strategy=1.0; total time= 1.2min
[CV] END model__colsample_bytree=0.741998522332775, model__learning_rate=0.008124072923574201, model__ma

In [ ]:
models = [
    'best_lgbm_smote_random',
    'best_lgbm_smote_20251106_155837',
    'best_lgbm_recall_optimized_20251106_162548'
]

In [9]:
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_recall_optimized_20251106_162548.pkl')
predictions = loaded_model.predict(X_test)

In [10]:
# 테스트셋 성능 평가
y_pred = loaded_model.predict(X_test)
y_prob = loaded_model.predict_proba(X_test)[:, 1]

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC (OvO, macro)
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
roc_auc_ovo = roc_auc_score(y_test_bin, y_prob, average='macro')
print(f"ROC-AUC (OvO, macro, Test Set): {roc_auc_ovo:.3f}")


Classification Report (Test Set):
              precision    recall  f1-score   support

           0      0.223     0.390     0.284      4698
           1      0.857     0.729     0.788     23600

    accuracy                          0.673     28298
   macro avg      0.540     0.560     0.536     28298
weighted avg      0.752     0.673     0.704     28298


Balanced Accuracy (Test Set): 0.560
ROC-AUC (OvO, macro, Test Set): 0.598
